# 가벼운 딥러닝 적용1 - REGRESSION

## 가벼운 REGRESSION 예제 문제

In [1]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler

In [2]:
df_train = pd.read_csv('house-prices-advanced-regression-techniques/train.csv')
df_test = pd.read_csv('house-prices-advanced-regression-techniques/test.csv')

In [3]:
print(df_train.shape, df_test.shape)

(1460, 81) (1459, 80)


In [4]:
df_train.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [6]:
df_test.head() # feature(정답)가 없음

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,...,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1,1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
2,1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
3,1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
4,1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,...,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal


In [9]:
df_train.corr(numeric_only=True)['SalePrice']

Id              -0.021917
MSSubClass      -0.084284
LotFrontage      0.351799
LotArea          0.263843
OverallQual      0.790982
OverallCond     -0.077856
YearBuilt        0.522897
YearRemodAdd     0.507101
MasVnrArea       0.477493
BsmtFinSF1       0.386420
BsmtFinSF2      -0.011378
BsmtUnfSF        0.214479
TotalBsmtSF      0.613581
1stFlrSF         0.605852
2ndFlrSF         0.319334
LowQualFinSF    -0.025606
GrLivArea        0.708624
BsmtFullBath     0.227122
BsmtHalfBath    -0.016844
FullBath         0.560664
HalfBath         0.284108
BedroomAbvGr     0.168213
KitchenAbvGr    -0.135907
TotRmsAbvGrd     0.533723
Fireplaces       0.466929
GarageYrBlt      0.486362
GarageCars       0.640409
GarageArea       0.623431
WoodDeckSF       0.324413
OpenPorchSF      0.315856
EnclosedPorch   -0.128578
3SsnPorch        0.044584
ScreenPorch      0.111447
PoolArea         0.092404
MiscVal         -0.021190
MoSold           0.046432
YrSold          -0.028923
SalePrice        1.000000
Name: SalePr

In [13]:
df_train.corr(numeric_only=True)['SalePrice'][(df_train.corr(numeric_only=True)['SalePrice']>0.5)|(df_train.corr(numeric_only=True)['SalePrice']<-0.5)]

OverallQual     0.790982
YearBuilt       0.522897
YearRemodAdd    0.507101
TotalBsmtSF     0.613581
1stFlrSF        0.605852
GrLivArea       0.708624
FullBath        0.560664
TotRmsAbvGrd    0.533723
GarageCars      0.640409
GarageArea      0.623431
SalePrice       1.000000
Name: SalePrice, dtype: float64

In [14]:
feature_list = df_train.corr(numeric_only=True)['SalePrice'][(df_train.corr(numeric_only=True)['SalePrice']>0.5)|(df_train.corr(numeric_only=True)['SalePrice']<-0.5)].index

In [15]:
df_train[feature_list].isnull().sum()

OverallQual     0
YearBuilt       0
YearRemodAdd    0
TotalBsmtSF     0
1stFlrSF        0
GrLivArea       0
FullBath        0
TotRmsAbvGrd    0
GarageCars      0
GarageArea      0
SalePrice       0
dtype: int64

In [16]:
df_train[feature_list].dtypes

OverallQual     int64
YearBuilt       int64
YearRemodAdd    int64
TotalBsmtSF     int64
1stFlrSF        int64
GrLivArea       int64
FullBath        int64
TotRmsAbvGrd    int64
GarageCars      int64
GarageArea      int64
SalePrice       int64
dtype: object

In [21]:
features = list(feature_list)[:-1]
features

['OverallQual',
 'YearBuilt',
 'YearRemodAdd',
 'TotalBsmtSF',
 '1stFlrSF',
 'GrLivArea',
 'FullBath',
 'TotRmsAbvGrd',
 'GarageCars',
 'GarageArea']

In [25]:
for feature in features:
    df_test[feature].fillna((df_test[feature].mean()), inplace=True)

C:\Users\JHN\AppData\Local\Temp\ipykernel_22784\909801948.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_test[feature].fillna((df_test[feature].mean()), inplace=True)


In [28]:
X_train = df_train[features] # 특정 feature 열만 뽑아옴
y_train = df_train[['SalePrice']].values # (n, 1)로 뽑고 values로 넘파이 배열로 변환시킴
X_test = df_test[features]

In [29]:
type(y_train)

numpy.ndarray

In [30]:
std_scaler = StandardScaler()
std_scaler.fit(X_train)
X_train_tensor = torch.from_numpy(std_scaler.transform(X_train)).float()
X_test_tensor = torch.from_numpy(std_scaler.transform(X_test)).float()
y_train_tensor = torch.from_numpy(y_train).float()

In [31]:
print(X_train_tensor.shape, X_test_tensor.shape, y_train_tensor.shape)

torch.Size([1460, 10]) torch.Size([1459, 10]) torch.Size([1460, 1])


In [32]:
nb_epochs = 10000
minibatch_size = 256

In [33]:
class FunModel(nn.Module):
    def __init__(self, input_dim, output_dim):
        super().__init__()

        self.linear_layers = nn.Sequential(
            nn.Linear(input_dim, 20),
            nn.LeakyReLU(),
            nn.Linear(20, 10),
            nn.LeakyReLU(),
            nn.Linear(10, 8),
            nn.LeakyReLU(),
            nn.Linear(8, 6),
            nn.LeakyReLU(),
            nn.Linear(6, output_dim)
        )

    def forward(self, x):
        y = self.linear_layers(x)
        return y

In [35]:
input_dim = X_train_tensor.size(-1)
output_dim = y_train_tensor.size(-1)
print(input_dim, output_dim)
model = FunModel(input_dim, output_dim)
loss_function = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters())

10 1


In [40]:
# indices = torch.randperm(X_train_tensor.size(0))
# print(indices)
# x_batch_list = torch.index_select(X_train_tensor, 0, index=indices)
# y_batch_list = torch.index_select(y_train_tensor, 0, index=indices)
# x_batch_list = x_batch_list.split(minibatch_size, 0)
# y_batch_list = y_batch_list.split(minibatch_size, 0)

In [43]:
for index in range(nb_epochs):
    indices = torch.randperm(X_train_tensor.size(0))

    x_batch_list = torch.index_select(X_train_tensor, 0, index=indices)
    y_batch_list = torch.index_select(y_train_tensor, 0, index=indices)
    x_batch_list = x_batch_list.split(minibatch_size, 0)
    y_batch_list = y_batch_list.split(minibatch_size, 0)

    epoch_loss = list()
    for x_minibatch, y_minibatch in zip(x_batch_list, y_batch_list):
        y_minibatch_pred = model(x_minibatch)

        loss = torch.sqrt(loss_function(y_minibatch_pred, y_minibatch))
        epoch_loss.append(loss)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    if (index % 100) == 0:
        print(index, sum(epoch_loss)/len(epoch_loss))

0 tensor(31880.2715, grad_fn=<DivBackward0>)
100 tensor(31420.6484, grad_fn=<DivBackward0>)
200 tensor(31466.6641, grad_fn=<DivBackward0>)
300 tensor(31671.5488, grad_fn=<DivBackward0>)
400 tensor(32024.7246, grad_fn=<DivBackward0>)
500 tensor(31183.1035, grad_fn=<DivBackward0>)
600 tensor(31766.7402, grad_fn=<DivBackward0>)
700 tensor(31185.6465, grad_fn=<DivBackward0>)
800 tensor(32043.1270, grad_fn=<DivBackward0>)
900 tensor(31926.6816, grad_fn=<DivBackward0>)
1000 tensor(31389.2031, grad_fn=<DivBackward0>)
1100 tensor(31674.1035, grad_fn=<DivBackward0>)
1200 tensor(31370.1035, grad_fn=<DivBackward0>)
1300 tensor(32069.9512, grad_fn=<DivBackward0>)
1400 tensor(31091.7578, grad_fn=<DivBackward0>)
1500 tensor(31428.0781, grad_fn=<DivBackward0>)
1600 tensor(31480.5312, grad_fn=<DivBackward0>)
1700 tensor(32198.9316, grad_fn=<DivBackward0>)
1800 tensor(30780.1172, grad_fn=<DivBackward0>)
1900 tensor(32172.0059, grad_fn=<DivBackward0>)
2000 tensor(31399.5234, grad_fn=<DivBackward0>)
2100

In [45]:
y_pred_list = list() # 테스트셋 예측값을 저장할 리스트 생성
x_test_batch_list = X_test_tensor.split(minibatch_size, 0) # 테스트셋은 인덱스 랜덤이 필요없기 때문에 미니 배치만

model.eval() # 특정 layer들을 수행하지 않음
with torch.no_grad(): # gradient 계산을 수행하지 않음
    for x_minibatch in x_test_batch_list:
        y_minibatch_pred = model(x_minibatch)
        y_pred_list.extend(y_minibatch_pred.squeeze().detach().tolist()) # 예측값 (256, 1)을 (256, )로 변경, 기존과 그래프 끊음, 리스트로 변환 후 extend로 계속 저장
print(y_pred_list)

# submission = pd.DataFrame({'Id': df_test['Id'], 'SalePrice': y_pred_list})
# submission.to_csv('submission.csv', index=False)

[132732.6875, 155674.53125, 168873.5, 176686.1875, 212257.0625, 177246.03125, 169517.21875, 164222.0625, 201598.46875, 117491.90625, 199863.53125, 109493.5703125, 102362.3515625, 156298.3125, 144794.328125, 386562.25, 272196.21875, 337857.84375, 304010.125, 429341.59375, 296481.4375, 227608.84375, 183951.59375, 185840.234375, 168403.921875, 203605.375, 312638.25, 267944.46875, 209660.78125, 199837.796875, 199608.875, 106529.65625, 183426.453125, 303730.9375, 290197.21875, 214856.59375, 219878.5, 172084.78125, 171562.0625, 160661.671875, 176781.59375, 161837.59375, 262990.25, 226819.046875, 210948.53125, 188055.96875, 246751.546875, 212011.984375, 158902.265625, 144716.4375, 143286.28125, 160599.28125, 154707.84375, 167264.8125, 202833.6875, 148774.578125, 157603.734375, 149626.28125, 216969.6875, 132089.140625, 136820.265625, 175941.59375, 106934.796875, 127307.3359375, 122278.9375, 123056.859375, 112884.578125, 130689.234375, 139622.78125, 164769.671875, 105297.7109375, 95937.953125, 